In [356]:
import pandas as pd													# data manipulation
import matplotlib.pyplot as plt												# graphs
import seaborn as sns													# graphs
import numpy as np													# graphs
from scipy.stats import norm												# statistics
from sklearn.preprocessing import StandardScaler									# statistics
from scipy import stats													# statistics
from math import *													# advance calculation	
from matplotlib import cm												# graphs
import sys														# system functions
import warnings														# indicating warnings
warnings.filterwarnings('ignore')											# ignore by default
from matplotlib.ticker import LinearLocator										# graphs
pd.set_option('display.max_columns', None)										# show all columns
pd.set_option('display.max_rows', None)											# show all rows
np.set_printoptions(threshold=sys.maxsize)										# show arrays full
pd.set_option("display.max_colwidth", -1)										# show a complete text in a cell
pd.set_option('display.precision',3)
from xlsxwriter.workbook import Workbook										# CSV writing
import csv														# CSV writing	
import xgboost as xgb													# prediction
import pickle														# prediction	
import sklearn														# mutual information
import statsmodels.api as sm												# coefficient
from sklearn.ensemble import RandomForestClassifier									# random forest
from sklearn import linear_model											# linear regression
import pymysql														# MySql connection
import datetime														# time conversions
import random														# randomchoice
import math as math													# maths
import statistics													# statistics
import glob														# paths and files management

In [518]:
# Snapp Pricing at driver's level

df = pd.read_csv('Snapp_Pricing.csv', skipinitialspace=True)

In [519]:

df.drop(columns=['Phone Number','Email','Fare Base','Payouts Cash Collected'], inplace=True)
df.columns=['Driver','Date','TripID','TripType','FareDistance','FareDropoff','FarePickup','FareCommission','FarePromotion',
            'FareTip','FareTotal']
cols=['FareDistance', 'FareDropoff','FarePickup', 'FareCommission', 'FarePromotion', 'FareTip','FareTotal']
for column in cols:
    df[column].replace({'ZAR\xa0':''}, regex=True, inplace=True)
    df[column] = df[column].astype(float)
    
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values(['Driver', 'Date'], ascending=[True, True])
df = df.reset_index(drop=True)
df['Day'] = df['Date'].dt.day_name()
df = df[['Driver','TripType', 'TripID', 'Date', 'Day', 'FareDistance', 'FareDropoff',
   'FarePickup', 'FareCommission', 'FarePromotion', 'FareTip','FareTotal']]
drop1=df.loc[(df['FareDistance'].isnull())|(df['FareDropoff'].isnull())|(df['FarePickup'].isnull())|(df['FareCommission'].isnull())]
df.drop(drop1.index,inplace=True)
df['FarePromotion'] = df['FarePromotion'].fillna(0)
df['FareTip'] = df['FareTip'].fillna(0)

In [520]:
# Commission per trip

round((df['FareCommission'].sum())/(df.shape[0]),2)

-3.21

In [521]:
# Commission per day

round((df['FareCommission'].sum())/((df['Date'].tail(1).reset_index()-df['Date'].head(1).reset_index())['Date']/np.timedelta64(1, 'D')).sum(),2)

-18.39

In [522]:
# Snapp revenue from driver in the given time

round((abs(df['FareCommission']) - df['FarePromotion']).sum(),2)

-0.21

In [523]:
# Percentage of income driven from tip to total

round((df['FareTip'].sum()/df['FareTotal'].sum())*100,2)

31.67

In [524]:
# Income per day after commission deduction

round(df['FareTotal'].sum()/((df['Date'].tail(1).reset_index()-df['Date'].head(1).reset_index())['Date']/np.timedelta64(1, 'D')).sum(),2)

268.98

In [550]:
# Productivity per day

period = [0] * 100
for day in df['Day'].unique():
    weekday = df.loc[df['Day'] == day]
    i=0
    for date in weekday['Date'].astype(str).str[:10].unique():
        weekday2 = weekday.loc[weekday['Date'].astype(str).str[:10] == date]
        period[i]=((weekday2['Date'].tail(1).reset_index()-weekday2['Date'].head(1).reset_index())['Date']/np.timedelta64(1, 'h')).sum()
        i=i+1  
    print(day,':',round((weekday['FareTotal'].sum())/(sum(period)),2))


Monday : 46.88
Tuesday : 63.63
Wednesday : 50.06
Thursday : 62.58
Friday : 94.29
Saturday : 77.25
Sunday : 64.36
